#### DA 231o Data Engineering at Scale (August 2023)
# Project : Web Server Log Analysis Using Spark

## version: 1

### Change Log
v0.1, 2023-09-06: Creation of the template


##Abstract:  
Web server log analysis can be a potential solution to various problems related to understanding and optimizing web based system and services. Some major problems for which web server analysis can include User Behaviour analysis, performance monitoring, E-commerce optimization, Ad Campaign effectiveness etc. This can also help in server monitoring and taking decision for service scale up/down. Thus, can save from getting startled in case of increased load into server.
The goal of the project is to perform comprehensive analysis of any information which contains data from http /http2 URLs which can be from any sector, e-commerce logs, telecom logs, finance logs etc.

---

## Problem Domain and Dataset

In Web domain of HTTP based server client architecture, most of the system suffers through different kind of attacks. This attacks can be of differente magnitude ranging from service unavailability, command execution, data leaking and malicious response from the server.
One such case we are trying to analyze based on the real world data set available to us. In this project we have tried to manifest the same scenario in telecom domain, where with the progress to 5G architecture this kind of situation is anticipated.

Though there are several techniques in the 5G architecture it self to address this issues, but when deployed at private network level still malicious attacks are common threat for any operator.


##Dataset

We have captured the HTTP based req/response for some of the vital network functions in 5G network. Mostly from different span.
Also we have a dataset which contains the suspicious attacks that have been detected at the as per the behavior of that particular device or channel.
So, with these dataset we will try to analyze the impact on each network function and segregate different kind of attacks. Also we will manifest the behavior of each network function during these attacks.

*** Network function are basically web server.

* **Attacks.csv**: Each row in this file contains details about the connections that were made to the servers. Some of the connection are harmless where as few are detected by the ingress controller as malicious. With this dataset we can analyze the impact on different core network functions.
```
-- ts|uid|id.orig_h|id.orig_p|id.resp_h|id.resp_p|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|label|detailed-label: string (nullable = true)
```
More details are  here: [Any external link]


* **server_behavior.csv**: This file contains the HTTP req/res with failure result code from the network functions at several spans. There can be erroneous response from the function based on different cases. But the 500 series of HTTP error code shall be on our focus. It reflects the imapct on the network function out attack like DOS and DDOS.
 ```|-- No : integer (nullable = true) ```

 ```|-- Time: string (nullable = true)```

 ```|-- Source: string (nullable = true)```

 ```|-- Destination: string (nullable = true)```

 ```|-- Source Port : string (nullable = true)```

 ```|-- Destination Port: string (nullable = true)```

 ```|-- Protocol: string (nullable = true)```

 ```|-- Length: integer (nullable = true)```

 ```|-- Status: string (nullable = true)```

 ```|-- Info: string (nullable = true)```


Install Spark/Hadoop and Python dependencies

In [1]:
#######################################
###!@0 START INIT ENVIRONMENT
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls -al /content/drive/Shareddrives/DA231-2023/assignments/spark-3.0.3-bin-hadoop2.7.tgz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf /content/drive/Shareddrives/DA231-2023/assignments/spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"
###!@0 END INIT ENVIRONMENT

Mounted at /content/drive
-r-------- 1 root root 220400553 Sep  4 09:38 /content/drive/Shareddrives/DA231-2023/assignments/spark-3.0.3-bin-hadoop2.7.tgz


 Standard initialization for the Spark session.

In [2]:
#######################################
###!@1 START OF PYSPARK INIT
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
input_type = 'sample'
spark = SparkSession.builder\
         .master("local")\
         .appName("MyApp")\
         .config('spark.ui.port', '4050')\
         .getOrCreate()
spark
# Spark is ready to go within Colab!
###!@1 END OF PYSPARK INIT

In [3]:
#######################################
###!@1 START OF PYSPARK INIT
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()

###!@1 END OF PYSPARK INIT

### Initialize all the variables

In [ ]:

#######################################
###!@2 START OF DEFINING INPUT FILES
basepath = "/content/drive/MyDrive/content/"
Attack_dataset = basepath + "generated_data.csv"
#Attack_dataset = basepath + "Attack_details.csv"
Server_status_dataset = basepath + "server_status.csv"
Maintenance_dataset = basepath + "maintanence_schedule.csv"

df_StatusData = spark.read.csv(Server_status_dataset, header=True, inferSchema=True)
df_AttackData = spark.read.csv(Attack_dataset, header=True, inferSchema=True)
df_MaintaData = spark.read.csv(Maintenance_dataset, header=True, inferSchema=True)
###!@3 END OF LOADING DATA

### Common Imports


In [ ]:
#######################################
###!@5 START COMMON USER IMPORTS
#######################################
## Specify valid imports, if any, common to ALL your answers  ==========
## start your edits here =================
import math
import time
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from pyspark.sql.functions import *
from matplotlib import pyplot as plt
import plotly.express as px
import numpy as np

## end your edits here ==============

### Utility APIs

In [ ]:
#######################################
###!@6 START COMMON USER FUNCTIONS
#######################################
## Specify user defined functions, if any, common to ALL your answers   =====
## start your edits here =================


def get_labels(AttackCleanedDF):
  return AttackCleanedDF.select("label").distinct().rdd.flatMap(lambda x: x).collect()






## end your edits here =================
###!@6 END COMMON USER FUNCTIONS

### ---
### **1) Data Parsing and Cleaning (DATA Munging)**

***Context:*** *Data Science in the wild requires data preparation and cleaning before you can perform exploratory analysis on them. This can include parsing, removing invalid fields, extrapolation, etc. Let's try a flavor of it here.*

### **Below is the output for raw data structure from dataset.**
## **Attach.CSV**
 ```
 |-- timestamp: string (nullable = true)
 |-- id_orig_h: string (nullable = true)
 |-- id_orig_p: string (nullable = true)
 |-- id_resp_h: string (nullable = true)
 |-- id_resp_p: string (nullable = true)
 |-- proto: string (nullable = true)
 |-- detailed-label: string (nullable = true)
 ```

### **Server_behavior.csv**
 ```
|-- No : integer (nullable = true)
# |-- Time: string (nullable = true)

*   List item
*   List item


|-- Source: string (nullable = true)
|-- Destination: string (nullable = true)
|-- Source Port : string (nullable = true)
|-- Destination Port: string (nullable = true)
|-- Protocol: string (nullable = true)
|-- Length: integer (nullable = true)
|-- Status: string (nullable = true)
|-- Info: string (nullable = true)
```

We need to perform DATA munging activity to clean and formalize the data for further data processing. So below code shall perform DATA munging activity.
Activity:
    1. Convert Attack.csv into csv.
    2. Remove any entries with empty source and destination IP port
    3. Convert the timestamp from UNIX format to readable format.
    4. Convert the column name to same format. Remove "." from column name.

In [ ]:
#######################################
###convert Attack dataset to csv format
data = spark.read.text(Attack_dataset)
header = data.first().value.split("|")
schema = StructType([StructField(name, StringType(), True) for name in header])

# Skip the first line (header) and split the remaining lines
lines =  data.rdd.zipWithIndex().map(lambda x: x[0].value.split("|")).filter(lambda row: len(row) == len(header))
# Create a DataFrame from the cleaned data and schema
df = spark.createDataFrame(lines, schema=schema)
print(df.count())

##convert the  ts fields from unix timestamp to readable format.
df = df.withColumn("timestamp", from_unixtime("ts"))
df = df.drop("ts")

##remove "." from the coulmn names
new_column_names = []
for column_name in df.columns:
    new_column_name = column_name.replace(".", "_")
    new_column_names.append(new_column_name)

#Rename the columns
for i, old_name in enumerate(df.columns):
    df = df.withColumnRenamed(old_name, new_column_names[i])

# Reorder the columns to have "timestamp" at the beginning
column_order = ["timestamp"] + [col_name for col_name in df.columns if col_name != "timestamp"]
df =df.select(column_order)

###Select the relevant column for our operation
AttackCleanedDF = df.select("timestamp", "id_orig_h", "id_orig_p", "id_resp_h", "id_resp_p", 'proto', 'duration', 'label')
#AttackCleanedDF.show()
#AttackCleanedDF.printSchema()

## No cleaning is required for Server_Details and Maintenance_schedule dataset

## end your edits here =================

### **2) Removal of BENIGN only sources from the data set**
a). Check if there is any source which later on turned into an attacker.

b). Remove sources with no attack from the dataset. Keep source with both (BENIGN and MALICIOUS) or only MALICIOUS role.



In [ ]:
#AttackCleanedDF.persist()
#AttackSampleDF = AttackCleanedDF.sample(False, 1.0)
#AttackCleanedDF.show(10)#select('id_orig_h').distinct().show(10)
#StatusSample = df_StatusData.sample(False, 0.1)
#df_StatusData.persist()
#AttackCleanedDF.show()
#df_StatusData.show()


##In this section we will see three types of sources
1. Pure Benign Source
2. Pure Malicious Source
3. Mixed Source



In [ ]:
!pip install -U kaleido
import plotly.io as pio

def getBenignSources(AttackCleanedDF):
  BenignSources = AttackCleanedDF.filter(AttackCleanedDF["label"].startswith('B')).select("id_orig_h").distinct()
  return BenignSources
def getMaliciousSources(AttackCleanedDF):
  MaliciousSources = AttackCleanedDF.filter(AttackCleanedDF["label"].startswith('M')).select("id_orig_h").distinct()
  return MaliciousSources
def getMixedSources(BenignSources, MaliciousSources):
  BenignMaliciousSources = BenignSources.join(MaliciousSources,BenignSources.id_orig_h ==  MaliciousSources.id_orig_h,"inner")\
                                        .drop(MaliciousSources.id_orig_h)\
                                        .groupBy("id_orig_h").count().select("id_orig_h")
  return BenignMaliciousSources
def plotAllSourceType(data,valueCol,nameCol,title='Title'):
  fig = px.pie(data.toPandas().to_dict(), values=valueCol, names=nameCol, title=title)
  fig.show()
  #pio.write_image(fig, "figname.png")
  #fig.save()


BenignSources = getBenignSources(AttackCleanedDF)
MaliciousSources = getMaliciousSources(AttackCleanedDF)
MixedSources = getMixedSources(BenignSources, MaliciousSources)
PureBenignSources = BenignSources.subtract(MixedSources).withColumn("type", lit("Pure Benign"))
PureMaliciousSources = MaliciousSources.subtract(MixedSources).withColumn("type", lit("Pure Malicious"))
MixedTypeSources = MixedSources.withColumn("type", lit("Mixed"))

#MixedTypeSources.show()
#PureMaliciousSources.show()
#MixedTypeSources.show()

UserTypeDf = MixedTypeSources.union(PureMaliciousSources).union(PureBenignSources)
UserTypeDf.show()

UserTypeCntDf = UserTypeDf.groupBy('type').count()
UserTypeCntDf.show()

plotAllSourceType(UserTypeCntDf,'count','type','Type Of Users')

In [ ]:
def plotMixedDFwithTime(MixedCleanedDF,BCleanedDF):
  #plot for 24 hour buckets
  d = MixedCleanedDF.toPandas().to_dict()
  b = BCleanedDF.toPandas().to_dict()

  hours = np.arange(24)
  benign = [int(0) for _ in range(24)]
  malicious = [int(0) for _ in range(24)]
  pb = [int(0) for _ in range(24)]
  width = 0.4
  for i in range(24):
    if i in d['count'].keys():
      count = d['count'][i]
      category = d['typeMB'][i]
      hour = d['hour'][i]
      if category == 'M':
        malicious[hour] += count
      else:
        benign[hour] += count
  for i in range(24):
    if i in b['count'].keys():
      count = b['count'][i]
      category = b['typeMB'][i]
      hour = b['hour'][i]
      if category == 'B':
        pb[hour] += count
  plt.bar(hours, malicious, width, label="Malicious")
  plt.bar(hours+width, benign, width, label="False Benign")
  #plt.bar(hours+2*width, pb, width, label="True Users")
  plt.title("Behaviour of Attackers Classified as Benign")
  plt.xticks(hours)
  plt.xlabel("Hour")
  plt.ylabel("Number of requests")
  plt.legend()
  plt.show()

#Create MaliciousDF and MixedDF
OnlyMixedTypeDF = UserTypeDf.filter(UserTypeDf["type"] == "Mixed")
MixedDF = AttackCleanedDF.join(OnlyMixedTypeDF,AttackCleanedDF.id_orig_h == OnlyMixedTypeDF.id_orig_h,"inner").drop(OnlyMixedTypeDF.id_orig_h)
#MixedDF.show()

#append hour column
MixedDFWithTime = MixedDF.withColumn("hour", hour(MixedDF.timestamp)).withColumn("typeMB",substring(MixedDF.label,0,1))
#MixedDFWithTime.show()

MixedCleanedDF = MixedDFWithTime.select("id_orig_h","typeMB","hour").groupBy("typeMB","hour").count()
MixedCleanedDF.show()

#Create MaliciousDF and MixedDF
OnlyBenignTypeDF = UserTypeDf.filter(UserTypeDf["type"] == "Pure Benign")
BDF = AttackCleanedDF.join(OnlyBenignTypeDF,AttackCleanedDF.id_orig_h == OnlyBenignTypeDF.id_orig_h,"inner").drop(OnlyBenignTypeDF.id_orig_h)
#MixedDF.show()

#append hour column
BDFWithTime = BDF.withColumn("hour", hour(BDF.timestamp)).withColumn("typeMB",substring(BDF.label,0,1))
#MixedDFWithTime.show()

BCleanedDF = BDFWithTime.select("id_orig_h","typeMB","hour").groupBy("typeMB","hour").count()
BCleanedDF.show()

plotMixedDFwithTime(MixedCleanedDF,BCleanedDF)

In [ ]:
from datetime import datetime
dt = datetime.now()
import time
time.time()

### **3) Identify all categories of attack from dataset.**

   a) Create pie chart to depict the categorization and percentage of these attcak.( number of occurrance)

   B) Create pie chart to depict the categorization and percentage of these attcak.(Duration)

   c) Create pie chart to depict the categorization and percentage of these attcak.(Bandwidth no of packets)


### We can see 4 types of Attacks in our systems.
1. Malicious   PartOfAHorizontalPortScan
2. Malicious   C&C
3. Malicious   DDoS
4. Malicious

In [ ]:
groupedMaliciousDF = AttackCleanedDF.filter(AttackCleanedDF.label.contains("Malicious")).groupBy("label").count()
groupedMaliciousDF.show()

plotAllSourceType(groupedMaliciousDF,'count','label','Types Of Attack')


**Conclusion**
- More than half of the attacks are DDoS

## **Category wise Average Duration of Attacks**

In [ ]:
groupedMaliciousDurationDF = AttackCleanedDF.filter(AttackCleanedDF.label.contains("Malicious"))\
                                            .filter(~AttackCleanedDF.duration.contains("-"))\
                                            .groupBy("label").agg(avg("duration").alias("avg_duration"))
#groupedMaliciousDurationDF.show()

plotAllSourceType(groupedMaliciousDurationDF,'avg_duration','label','Average Duration of Attacks')

# **Summary**
- All Types Of Attacks have similar Duration. They are lasting for Avg 3 Min Duration.



### **4. Analyze the server behavior during the time of attack.**

   a) Categorize diferent kind of servers from the logs.

   b) Create pie chart to depict the impact caused by different types of attack on these servers.

   c) Which server was mostly impacted because of these attack?

   D) Analyze the type of imapct casued in these servers using HTTP return code.

   E) capture the behavior in Pie chart or similar diagram.


In [ ]:
#df_StatusData.show()
df_copy = df_StatusData.alias("df_copy")

def categorize_info(info):
    return when(info.contains('nausf-auth'), 'AUSF') \
           .when(info.contains('nudm-sdm'), 'UDM') \
           .when(info.contains('npcf-smpolicy'), 'PCF') \
           .otherwise('Responses')

# Apply the categorize_info function to create a new 'Category' column
df_copy = df_copy.withColumn('server_type', categorize_info(col('Info')))
# Count the occurrences for each category
category_counts = df_copy.groupBy('server_type').agg(count('*').alias('Count'))

# Filter out rows with 'Other' category
category_counts = category_counts.filter(col('server_type').isin('AUSF', 'UDM', 'PCF'))
# Show the resulting counts
#category_counts.show()

#df_copy.show(truncate=False)

#create a pie chart
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


def draw_pie_chart(pd_df, column_name, title):
    pd_df = category_counts.toPandas()
    plt.figure(figsize=(8, 8))
    plt.pie(pd_df['Count'], labels=pd_df[column_name], autopct='%1.1f%%', startangle=140)
    plt.title(title)
    plt.show()

draw_pie_chart(category_counts, 'server_type', 'Failure Distribution')




df_copy = df_StatusData.alias("df_copy")

def categorize_info(info):
    return when(info.contains('500'), '500') \
           .when(info.contains('501'), '501') \
           .when(info.contains('502'), '502') \
           .when(info.contains('503'), '503') \
           .when(info.contains('511'), '511') \
           .when(info.contains('5'), 'HTTP ERROR') \
           .otherwise('Request')

# Apply the categorize_info function to create a new 'Category' column
df_copy = df_copy.withColumn('error_type', categorize_info(col('Info')))
# Count the occurrences for each category
category_counts = df_copy.groupBy('error_type').agg(count('*').alias('Count'))


# Filter out rows with 'Other' category
category_counts = category_counts.filter(col('error_type') != 'Request')
draw_pie_chart(category_counts, 'error_type', 'Error Distribution')

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
# Create a feature vector

OnlyMTypeDF = UserTypeDf.filter(UserTypeDf["type"] == "Pure Malicious")
MDF = AttackCleanedDF.join(OnlyMTypeDF,AttackCleanedDF.id_orig_h == OnlyMTypeDF.id_orig_h,"inner").drop(OnlyMTypeDF.id_orig_h)

categorical_columns = ["id_orig_h", "id_orig_p", "id_resp_h", "id_resp_p","proto"]
# Create StringIndexer instances for each column
string_indexers = StringIndexer(inputCol="id_orig_p", outputCol="id_orig_p_num")
indexed_df = string_indexers.fit(MDF).transform(MDF)
indexed_df = indexed_df.drop("id_orig_p")


string_indexers = StringIndexer(inputCol="id_orig_h", outputCol="id_orig_h_num")
indexed_df = string_indexers.fit(indexed_df).transform(indexed_df)
indexed_df = indexed_df.drop("id_orig_h")

string_indexers = StringIndexer(inputCol="id_resp_h", outputCol="id_resp_h_num")
indexed_df = string_indexers.fit(indexed_df).transform(indexed_df)
indexed_df = indexed_df.drop("id_resp_h")

string_indexers = StringIndexer(inputCol="id_resp_p", outputCol="id_resp_p_num")
indexed_df = string_indexers.fit(indexed_df).transform(indexed_df)
indexed_df = indexed_df.drop("id_resp_p")

string_indexers = StringIndexer(inputCol="proto", outputCol="proto_num")
indexed_df = string_indexers.fit(indexed_df).transform(indexed_df)
indexed_df = indexed_df.drop("proto")

string_indexers = StringIndexer(inputCol="label", outputCol="label_num")
indexed_df = string_indexers.fit(indexed_df).transform(indexed_df)
indexed_df = indexed_df.drop("label")

# Show the result
indexed_df.show()

In [ ]:
indexed_df.show()
indexed_df.printSchema()
indexed_df = indexed_df.filter(indexed_df.duration != 'duration')
indexed_df.select('id_orig_p_num').show()

In [ ]:
indexed_df.select('id_orig_p_num').distinct().show()
indexed_df.select('id_orig_h_num').distinct().show()
indexed_df.select('id_resp_p_num').distinct().show()
indexed_df.select('id_resp_p_num').distinct().show()
indexed_df.select('proto_num').distinct().show()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col
from pyspark.ml.linalg import DenseVector, SparseVector, Vectors, VectorUDT
from pyspark.sql import functions as F
from pyspark.sql.types import *




# Create a feature vector
feature_cols = ["id_orig_h_num", "id_orig_p_num", "id_resp_h_num", "id_resp_p_num", 'proto_num']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
indexed_df1 = assembler.transform(indexed_df)


toDense = lambda v: Vectors.dense(v.toArray())
toDenseUdf = F.udf(toDense, VectorUDT())
indexed_df1 = indexed_df1.withColumn('features', toDenseUdf('features'))
indexed_df1.select('features').show()

# Split the data into training and test sets (80% training, 20% testing)
(training_data, test_data) = indexed_df1.randomSplit([0.7, 0.3], seed=123)

training_data.printSchema()

# Initialize the Random Forest classifier
rf = RandomForestClassifier(labelCol="label_num", featuresCol="features", numTrees=10)

# Create a pipeline
pipeline = Pipeline(stages=[rf])

# Train the model
model = pipeline.fit(training_data)
# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label_num", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

### **5. Capture the outliers from the Server dataset and find the impact on these due maintenance operation at network level.**
``` Maintenance Dataset contains the operation details executed on these servers by operator.```

```TODO capture the fields of Maintenance dataset```

  a). Capture if there is any instance when server performance deteriorated, But the is no attack on it.

  b). Verify if the poor performance can be attributed to the scheduled maintenance from OPERATIONS.csv dataset.


### **6. Benefit of using protection mechanism like Authentication at Network level**
a) Check if any server able to identify and reject the attack request. (HTTP response with 511 Network Authentication Required)